In [35]:
# Standard library imports
import importlib
import math
import os
import string
from itertools import product

# Third-party imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from plotly.subplots import make_subplots
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# OBP imports
from obp.dataset import OpenBanditDataset
from obp.ope import (
    DirectMethod as DM_OBP,
    DoublyRobust as DR_OBP,
    InverseProbabilityWeighting as IPW,
    OffPolicyEvaluation,
    RegressionModel
)
from obp.policy import IPWLearner

# Local imports
import stats
import visualizations
importlib.reload(stats)
importlib.reload(visualizations)
from visualizations import (
    plot_bar_chart,
    plot_boxplot_with_stats,
    plot_distribution_with_cdf,
    plot_histogram_with_stats
)
from stats import (
    calculate_distribution_stats,
    compute_feature_combinations,
    compute_item_feature_distribution,
    compute_item_propensity_stats,
    compute_manual_propensity,
    compute_propensity_variance
)

# Pandas display options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

print(os.getcwd())

/Users/armandoordoricadelatorre/Documents/U of T/PhD/PhD Research/OBP_Replication


### Helper functions

In [52]:


def remap_user_features(df, feature_cols):
    """
    Map hash values in user_feature_N to short readable codes like A1, B1, ...
    """
    df_copy = df.copy()
    mapping_dicts = {}

    for col in feature_cols:
        # Extract the feature index (N from 'user_feature_N')
        feature_idx = col.split("_")[-1]
        uniques = df[col].dropna().unique()

        # Build codes A{N}, B{N}, C{N}...
        codes = [f"{letter}{feature_idx}" for letter in string.ascii_uppercase[:len(uniques)]]
        mapping = dict(zip(uniques, codes))

        df_copy[col] = df[col].map(mapping)
        mapping_dicts[col] = mapping

    return df_copy, mapping_dicts





### Import logged data from `all.csv` and `item_context.csv`

In [53]:

# BTS / ALL sample
log_df   = pd.read_csv("zr-obp/obd/bts/all/all.csv", index_col=0)
items_df = pd.read_csv("zr-obp/obd/bts/all/item_context.csv", index_col=0)

print("log_df shape:", log_df.shape)
print("items_df shape:", items_df.shape)

print("\nlog_df columns:")
print(log_df.columns.tolist()[:40])  # peek first ~40 col names

print("\nfirst 5 log rows:")
print(log_df.head())

print("\nfirst 5 item rows:")
print(items_df.head())

log_df shape: (10000, 89)
items_df shape: (80, 5)

log_df columns:
['timestamp', 'item_id', 'position', 'click', 'propensity_score', 'user_feature_0', 'user_feature_1', 'user_feature_2', 'user_feature_3', 'user-item_affinity_0', 'user-item_affinity_1', 'user-item_affinity_2', 'user-item_affinity_3', 'user-item_affinity_4', 'user-item_affinity_5', 'user-item_affinity_6', 'user-item_affinity_7', 'user-item_affinity_8', 'user-item_affinity_9', 'user-item_affinity_10', 'user-item_affinity_11', 'user-item_affinity_12', 'user-item_affinity_13', 'user-item_affinity_14', 'user-item_affinity_15', 'user-item_affinity_16', 'user-item_affinity_17', 'user-item_affinity_18', 'user-item_affinity_19', 'user-item_affinity_20', 'user-item_affinity_21', 'user-item_affinity_22', 'user-item_affinity_23', 'user-item_affinity_24', 'user-item_affinity_25', 'user-item_affinity_26', 'user-item_affinity_27', 'user-item_affinity_28', 'user-item_affinity_29', 'user-item_affinity_30']

first 5 log rows:
           

### Remapping categorical features to readable categories

In [54]:
user_feature_cols = [c for c in log_df.columns if c.startswith("user_feature")]
log_df_readable, mappings = remap_user_features(log_df, user_feature_cols)

print("Sample remapped features:")
display(log_df_readable[user_feature_cols].head())

print("\nMappings used:")
for feat, mapping in mappings.items():
    print(f"{feat}: {mapping}")

Sample remapped features:


,user_feature_0,user_feature_1,user_feature_2,user_feature_3
0,A0,A1,A2,A3
1,A0,B1,B2,B3
2,A0,A1,C2,A3
3,A0,A1,A2,B3
4,A0,A1,C2,B3



Mappings used:
user_feature_0: {'81ce123cbb5bd8ce818f60fb3586bba5': 'A0', 'cef3390ed299c09874189c387777674a': 'B0', '4ae385d792f81dde128124a925a830de': 'C0'}
user_feature_1: {'03a5648a76832f83c859d46bc06cb64a': 'A1', '2d03db5543b14483e52d761760686b64': 'B1', '6ff54aa8ff7a9dde75161c20a3ee4231': 'C1', 'f1c2d6a32ec39249160cf784b63f4c6f': 'D1', '8b50621825ffd909dd8d8317d366271f': 'E1'}
user_feature_2: {'7bc94a2da491829b777c49c4b5e480f2': 'A2', '2723d2eb8bba04e0362098011fa3997b': 'B2', 'c2e4f76cdbabecd33b8c762aeef386b3': 'C2', '719dab53a7560218a9d1f96b25d6fa32': 'D2', '9b2d331c329ceb74d3dcfb48d8798c78': 'E2', '302deff13f835d731df1c842eed95971': 'F2', '9f4e8271d3d3014af5f35124c2de5082': 'G2', '7ae37150e596e6e8f19e27a06bd4d359': 'H2', 'c7cce49040b6630e9b5484dfcc0e6cd1': 'I2'}
user_feature_3: {'c39b0c7dd5d4eb9a18e7db6ba2f258f8': 'A3', '9bde591ffaab8d54c457448e4dca6f53': 'B3', '05b76f5e97e51128862059ac7df9e42a': 'C3', 'f97571b9c14a786aab269f0b427d2a85': 'D3', '06128286bcc64b6a4b0fb7bc0328fe17'

In [55]:
log_df_readable.head()

,timestamp,item_id,position,click,propensity_score,user_feature_0,user_feature_1,user_feature_2,user_feature_3,user-item_affinity_0,user-item_affinity_1,user-item_affinity_2,user-item_affinity_3,user-item_affinity_4,user-item_affinity_5,user-item_affinity_6,user-item_affinity_7,user-item_affinity_8,user-item_affinity_9,user-item_affinity_10,user-item_affinity_11,user-item_affinity_12,user-item_affinity_13,user-item_affinity_14,user-item_affinity_15,user-item_affinity_16,user-item_affinity_17,user-item_affinity_18,user-item_affinity_19,user-item_affinity_20,user-item_affinity_21,user-item_affinity_22,user-item_affinity_23,user-item_affinity_24,user-item_affinity_25,user-item_affinity_26,user-item_affinity_27,user-item_affinity_28,user-item_affinity_29,user-item_affinity_30,user-item_affinity_31,user-item_affinity_32,user-item_affinity_33,user-item_affinity_34,user-item_affinity_35,user-item_affinity_36,user-item_affinity_37,user-item_affinity_38,user-item_affinity_39,user-item_affinity_40,user-item_affinity_41,user-item_affinity_42,user-item_affinity_43,user-item_affinity_44,user-item_affinity_45,user-item_affinity_46,user-item_affinity_47,user-item_affinity_48,user-item_affinity_49,user-item_affinity_50,user-item_affinity_51,user-item_affinity_52,user-item_affinity_53,user-item_affinity_54,user-item_affinity_55,user-item_affinity_56,user-item_affinity_57,user-item_affinity_58,user-item_affinity_59,user-item_affinity_60,user-item_affinity_61,user-item_affinity_62,user-item_affinity_63,user-item_affinity_64,user-item_affinity_65,user-item_affinity_66,user-item_affinity_67,user-item_affinity_68,user-item_affinity_69,user-item_affinity_70,user-item_affinity_71,user-item_affinity_72,user-item_affinity_73,user-item_affinity_74,user-item_affinity_75,user-item_affinity_76,user-item_affinity_77,user-item_affinity_78,user-item_affinity_79
0,2019-11-24 00:00:17.004101+00:00,79,2,0,0.087125,A0,A1,A2,A3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-11-24 00:00:19.715857+00:00,14,1,0,0.006235,A0,B1,B2,B3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-11-24 00:01:04.303227+00:00,18,2,0,0.061300,A0,A1,C2,A3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-11-24 00:01:11.571162+00:00,28,1,0,0.019430,A0,A1,A2,B3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-11-24 00:02:41.811768+00:00,65,2,0,0.019375,A0,A1,C2,B3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## EDA 

### Counting user features vs user-item features

In [56]:
user_feature_cols = [c for c in log_df_readable.columns if c.startswith('user_feature_')]
affinity_cols = [c for c in log_df_readable.columns if c.startswith('user-item_affinity_')]

user_feature_cols, len(affinity_cols)

(['user_feature_0', 'user_feature_1', 'user_feature_2', 'user_feature_3'], 80)

In [57]:
# Example: Use the function for item_id=0 and user_feature_0
temp_df_item_0 = compute_item_feature_distribution(log_df_readable, item_id=0, feature_col='user_feature_0')
temp_df_item_0

,user_feature_0,count,total_occurrences,proportion
0,A0,94,125,0.752
1,B0,31,125,0.248


In [58]:
# Example 1: Compute manual propensity for item_id and user_feature_0
manual_prop_uf0 = compute_manual_propensity(log_df_readable, categorical_col='user_feature_0')
print("Manual propensity by item_id and user_feature_0:")
display(manual_prop_uf0.head(10))
print(f"\nTotal rows: {len(manual_prop_uf0)}")

Manual propensity by item_id and user_feature_0:


,item_id,user_feature_0,count_of_occurrences,total,manual_propensity
0,0,A0,94,10000,0.0094
1,0,B0,31,10000,0.0031
2,1,A0,24,10000,0.0024
3,1,B0,7,10000,0.0007
4,2,A0,14,10000,0.0014
5,2,B0,3,10000,0.0003
6,3,A0,42,10000,0.0042
7,3,B0,4,10000,0.0004
8,4,A0,13,10000,0.0013
9,4,B0,3,10000,0.0003



Total rows: 187


In [59]:
item_id_0_manual_propensity_global_feature_0 =log_df_readable.groupby(['item_id', 'user_feature_0'])['propensity_score'].mean().reset_index()
item_id_0_manual_propensity_global_feature_0 = item_id_0_manual_propensity_global_feature_0[item_id_0_manual_propensity_global_feature_0['item_id'] == 0]
item_id_0_manual_propensity_global_feature_0

,item_id,user_feature_0,propensity_score
0,0,A0,0.054190
1,0,B0,0.040818


In [60]:
log_df_readable[(log_df_readable['item_id'] == 0) & (log_df_readable['user_feature_0'] == 'A0')]['propensity_score'].mean()

0.05419

In [61]:
log_df_readable[(log_df_readable['item_id'] == 0) & (log_df_readable['user_feature_0'] == 'B0')]['propensity_score'].mean()

0.040818225806451607

In [62]:
log_df_readable[(log_df_readable['item_id'] == 0)]['click'].mean()

0.0

In [63]:
log_df_readable[(log_df_readable['item_id'] == 49)]['click'].mean()

0.0024509803921568627

### Finding Missing User Feature Combinations for Item 0

Let's identify which user feature combinations have NOT been observed for item_id = 0

In [64]:
# Get all unique values for each user feature and calculate total combinations
unique_values, total_combinations = compute_feature_combinations(
    log_df_readable, 
    user_feature_cols, 
    verbose=True
)

Unique values per feature:
  user_feature_0: ['A0', 'B0', 'C0'] (n=3)
  user_feature_1: ['A1', 'B1', 'C1', 'D1', 'E1'] (n=5)
  user_feature_2: ['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2'] (n=9)
  user_feature_3: ['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3', 'I3'] (n=9)

Total possible feature combinations: 1,215


In [89]:
def get_missing_combinations(df, feature_cols, unique_values, item_id):
    item_N_data = df[df['item_id'] == item_id]
    observed_combos = item_N_data[feature_cols].drop_duplicates()

    print(f"Observed combinations for item_id={item_id}: {len(observed_combos)}")
    print("\nFirst 10 observed combinations:")
    print(observed_combos.head(10))

    # Create all possible combinations
    all_possible = list(product(*[unique_values[col] for col in feature_cols]))
    all_possible_df = pd.DataFrame(all_possible, columns=feature_cols)

    # Find missing combinations (anti-join)
    merged = all_possible_df.merge(
        observed_combos,
        on=feature_cols,
        how='left',
        indicator=True
    )

    missing_combos = merged[merged['_merge'] == 'left_only'][feature_cols]

    print(f"Missing combinations for item_id={item_id}: {len(missing_combos):,}")
    print(f"Coverage: {len(observed_combos)/len(all_possible_df)*100:.2f}%")

    print("\nFirst 10 missing combinations:")
    return missing_combos


missing_combinations_item49 = get_missing_combinations(log_df_readable, user_feature_cols, unique_values, 49)
missing_combinations_item49

Observed combinations for item_id=49: 95

First 10 observed combinations:
    user_feature_0 user_feature_1 user_feature_2 user_feature_3
34              A0             A1             E2             A3
70              A0             A1             E2             B3
242             A0             A1             A2             B3
516             A0             A1             C2             E3
730             A0             B1             A2             E3
819             A0             A1             B2             B3
829             A0             A1             B2             C3
858             A0             A1             C2             A3
859             A0             A1             F2             B3
860             A0             A1             B2             A3
Missing combinations for item_id=49: 1,120
Coverage: 7.82%

First 10 missing combinations:


,user_feature_0,user_feature_1,user_feature_2,user_feature_3
2,A0,A1,A2,C3
5,A0,A1,A2,F3
6,A0,A1,A2,G3
7,A0,A1,A2,H3
8,A0,A1,A2,I3
...,...,...,...,...
1210,C0,E1,I2,E3
1211,C0,E1,I2,F3
1212,C0,E1,I2,G3
1213,C0,E1,I2,H3


In [92]:
missing_combinations_item49.iloc[0].user_feature_0

'A0'

### Proving that this specific combination is indeed missing from the logs

In [93]:
log_df_readable_item49 = log_df_readable[log_df_readable['item_id'] == 49]
log_df_readable_item49[(log_df_readable_item49['user_feature_0'] == missing_combinations_item49.iloc[0].user_feature_0) &\
                       (log_df_readable_item49['user_feature_1'] == missing_combinations_item49.iloc[0].user_feature_1) &\
                       (log_df_readable_item49['user_feature_2'] == missing_combinations_item49.iloc[0].user_feature_2) &\
                       (log_df_readable_item49['user_feature_3'] == missing_combinations_item49.iloc[0].user_feature_3)]

,timestamp,item_id,position,click,propensity_score,user_feature_0,user_feature_1,user_feature_2,user_feature_3,user-item_affinity_0,user-item_affinity_1,user-item_affinity_2,user-item_affinity_3,user-item_affinity_4,user-item_affinity_5,user-item_affinity_6,user-item_affinity_7,user-item_affinity_8,user-item_affinity_9,user-item_affinity_10,user-item_affinity_11,user-item_affinity_12,user-item_affinity_13,user-item_affinity_14,user-item_affinity_15,user-item_affinity_16,user-item_affinity_17,user-item_affinity_18,user-item_affinity_19,user-item_affinity_20,user-item_affinity_21,user-item_affinity_22,user-item_affinity_23,user-item_affinity_24,user-item_affinity_25,user-item_affinity_26,user-item_affinity_27,user-item_affinity_28,user-item_affinity_29,user-item_affinity_30,user-item_affinity_31,user-item_affinity_32,user-item_affinity_33,user-item_affinity_34,user-item_affinity_35,user-item_affinity_36,user-item_affinity_37,user-item_affinity_38,user-item_affinity_39,user-item_affinity_40,user-item_affinity_41,user-item_affinity_42,user-item_affinity_43,user-item_affinity_44,user-item_affinity_45,user-item_affinity_46,user-item_affinity_47,user-item_affinity_48,user-item_affinity_49,user-item_affinity_50,user-item_affinity_51,user-item_affinity_52,user-item_affinity_53,user-item_affinity_54,user-item_affinity_55,user-item_affinity_56,user-item_affinity_57,user-item_affinity_58,user-item_affinity_59,user-item_affinity_60,user-item_affinity_61,user-item_affinity_62,user-item_affinity_63,user-item_affinity_64,user-item_affinity_65,user-item_affinity_66,user-item_affinity_67,user-item_affinity_68,user-item_affinity_69,user-item_affinity_70,user-item_affinity_71,user-item_affinity_72,user-item_affinity_73,user-item_affinity_74,user-item_affinity_75,user-item_affinity_76,user-item_affinity_77,user-item_affinity_78,user-item_affinity_79


In [94]:
log_df_readable.head()

,timestamp,item_id,position,click,propensity_score,user_feature_0,user_feature_1,user_feature_2,user_feature_3,user-item_affinity_0,user-item_affinity_1,user-item_affinity_2,user-item_affinity_3,user-item_affinity_4,user-item_affinity_5,user-item_affinity_6,user-item_affinity_7,user-item_affinity_8,user-item_affinity_9,user-item_affinity_10,user-item_affinity_11,user-item_affinity_12,user-item_affinity_13,user-item_affinity_14,user-item_affinity_15,user-item_affinity_16,user-item_affinity_17,user-item_affinity_18,user-item_affinity_19,user-item_affinity_20,user-item_affinity_21,user-item_affinity_22,user-item_affinity_23,user-item_affinity_24,user-item_affinity_25,user-item_affinity_26,user-item_affinity_27,user-item_affinity_28,user-item_affinity_29,user-item_affinity_30,user-item_affinity_31,user-item_affinity_32,user-item_affinity_33,user-item_affinity_34,user-item_affinity_35,user-item_affinity_36,user-item_affinity_37,user-item_affinity_38,user-item_affinity_39,user-item_affinity_40,user-item_affinity_41,user-item_affinity_42,user-item_affinity_43,user-item_affinity_44,user-item_affinity_45,user-item_affinity_46,user-item_affinity_47,user-item_affinity_48,user-item_affinity_49,user-item_affinity_50,user-item_affinity_51,user-item_affinity_52,user-item_affinity_53,user-item_affinity_54,user-item_affinity_55,user-item_affinity_56,user-item_affinity_57,user-item_affinity_58,user-item_affinity_59,user-item_affinity_60,user-item_affinity_61,user-item_affinity_62,user-item_affinity_63,user-item_affinity_64,user-item_affinity_65,user-item_affinity_66,user-item_affinity_67,user-item_affinity_68,user-item_affinity_69,user-item_affinity_70,user-item_affinity_71,user-item_affinity_72,user-item_affinity_73,user-item_affinity_74,user-item_affinity_75,user-item_affinity_76,user-item_affinity_77,user-item_affinity_78,user-item_affinity_79
0,2019-11-24 00:00:17.004101+00:00,79,2,0,0.087125,A0,A1,A2,A3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-11-24 00:00:19.715857+00:00,14,1,0,0.006235,A0,B1,B2,B3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-11-24 00:01:04.303227+00:00,18,2,0,0.061300,A0,A1,C2,A3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-11-24 00:01:11.571162+00:00,28,1,0,0.019430,A0,A1,A2,B3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-11-24 00:02:41.811768+00:00,65,2,0,0.019375,A0,A1,C2,B3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [97]:
# Random Forest Model to Predict Click Probability
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import pandas as pd
import numpy as np

print("="*60)
print("RANDOM FOREST MODEL FOR CLICK PREDICTION")
print("="*60)

# 1. Prepare the features and target
feature_cols = ['user_feature_0', 'user_feature_1', 'user_feature_2', 'user_feature_3', 'item_id']
target_col = 'click'

# Check data availability
print(f"Total rows in dataset: {len(log_df_readable):,}")
print(f"Features: {feature_cols}")
print(f"Target: {target_col}")
print(f"Target distribution:")
print(log_df_readable[target_col].value_counts())
print(f"Click rate: {log_df_readable[target_col].mean():.4f}")

# 2. Encode categorical features
print("\n" + "="*40)
print("ENCODING CATEGORICAL FEATURES")
print("="*40)

# Create label encoders for categorical features
label_encoders = {}
X_encoded = log_df_readable[feature_cols].copy()

for col in feature_cols:
    if col != 'item_id':  # item_id is already numeric
        le = LabelEncoder()
        X_encoded[col] = le.fit_transform(log_df_readable[col].astype(str))
        label_encoders[col] = le
        print(f"{col}: {len(le.classes_)} unique values")
        print(f"  Sample mapping: {dict(zip(le.classes_[:5], le.transform(le.classes_[:5])))}")

y = log_df_readable[target_col]

print(f"\nFinal feature matrix shape: {X_encoded.shape}")
print(f"Target vector shape: {y.shape}")

# 3. Train-test split
print("\n" + "="*40)
print("TRAIN-TEST SPLIT")
print("="*40)

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape[0]:,} samples")
print(f"Test set: {X_test.shape[0]:,} samples")
print(f"Train click rate: {y_train.mean():.4f}")
print(f"Test click rate: {y_test.mean():.4f}")

# 4. Train Random Forest
print("\n" + "="*40)
print("TRAINING RANDOM FOREST")
print("="*40)

rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)

# 5. Model evaluation
print("\nModel Performance:")
train_score = rf_model.score(X_train, y_train)
test_score = rf_model.score(X_test, y_test)

print(f"Train Accuracy: {train_score:.4f}")
print(f"Test Accuracy: {test_score:.4f}")

# Predictions for AUC
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]
test_auc = roc_auc_score(y_test, y_pred_proba)
print(f"Test AUC: {test_auc:.4f}")

# Feature importance
print(f"\nFeature Importances:")
for col, importance in zip(feature_cols, rf_model.feature_importances_):
    print(f"  {col:15s}: {importance:.4f}")

print("\n" + "="*60)
print("PREDICTION FOR TARGET COMBINATION")
print("="*60)

RANDOM FOREST MODEL FOR CLICK PREDICTION
Total rows in dataset: 10,000
Features: ['user_feature_0', 'user_feature_1', 'user_feature_2', 'user_feature_3', 'item_id']
Target: click
Target distribution:
0    9958
1      42
Name: click, dtype: int64
Click rate: 0.0042

ENCODING CATEGORICAL FEATURES
user_feature_0: 3 unique values
  Sample mapping: {'A0': 0, 'B0': 1, 'C0': 2}
user_feature_1: 5 unique values
  Sample mapping: {'A1': 0, 'B1': 1, 'C1': 2, 'D1': 3, 'E1': 4}
user_feature_2: 9 unique values
  Sample mapping: {'A2': 0, 'B2': 1, 'C2': 2, 'D2': 3, 'E2': 4}
user_feature_3: 9 unique values
  Sample mapping: {'A3': 0, 'B3': 1, 'C3': 2, 'D3': 3, 'E3': 4}

Final feature matrix shape: (10000, 5)
Target vector shape: (10000,)

TRAIN-TEST SPLIT
Training set: 8,000 samples
Test set: 2,000 samples
Train click rate: 0.0043
Test click rate: 0.0040

TRAINING RANDOM FOREST

Model Performance:
Train Accuracy: 0.9958
Test Accuracy: 0.9960
Test AUC: 0.3851

Feature Importances:
  user_feature_0 : 0.

In [100]:
# Predict for the specific combination
target_combination = {
    'user_feature_0': 'A0',
    'user_feature_1': 'A1', 
    'user_feature_2': 'A2',
    'user_feature_3': 'C3',
    'item_id': 49
}

print(f"Target combination to predict:")
for feature, value in target_combination.items():
    print(f"  {feature}: {value}")

# Check if this combination exists in the training data
exists_in_data = log_df_readable[
    (log_df_readable['user_feature_0'] == target_combination['user_feature_0']) &
    (log_df_readable['user_feature_1'] == target_combination['user_feature_1']) &
    (log_df_readable['user_feature_2'] == target_combination['user_feature_2']) &
    (log_df_readable['user_feature_3'] == target_combination['user_feature_3']) &
    (log_df_readable['item_id'] == target_combination['item_id'])
]

print(f"\nCombination exists in data: {len(exists_in_data)} observations")
if len(exists_in_data) > 0:
    observed_ctr = exists_in_data['click'].mean()
    print(f"Observed CTR: {observed_ctr:.4f}")
else:
    print("This is an UNSEEN combination - model will extrapolate")

# Encode the target combination
target_encoded = {}
for feature, value in target_combination.items():
    if feature in label_encoders:
        try:
            target_encoded[feature] = label_encoders[feature].transform([str(value)])[0]
        except ValueError:
            print(f"Warning: {value} not seen in training for {feature}")
            # Use the most common value as fallback
            target_encoded[feature] = 0
    else:
        # item_id is numeric
        target_encoded[feature] = value

print(f"\nEncoded values:")
for feature, encoded_val in target_encoded.items():
    print(f"  {feature}: {target_combination[feature]} -> {encoded_val}")

# Create prediction input
X_target = pd.DataFrame([target_encoded])
print(f"\nPrediction input shape: {X_target.shape}")
print(f"Prediction input:\n{X_target}")

# Make prediction
click_probability = rf_model.predict_proba(X_target)[0, 1]
click_prediction = rf_model.predict(X_target)[0]

print("\n" + "="*50)
print("FINAL PREDICTION RESULTS")
print("="*50)
print(f"Predicted click probability: {click_probability:.6f}")
print(f"Predicted class (click): {click_prediction}")
print(f"Confidence: {max(rf_model.predict_proba(X_target)[0]):.6f}")

# Compare with baseline rates
overall_ctr = log_df_readable['click'].mean()
item_49_ctr = log_df_readable[log_df_readable['item_id'] == 49]['click'].mean()

print(f"\nComparison with baselines:")
print(f"Overall CTR: {overall_ctr:.6f}")
print(f"Item 49 CTR: {item_49_ctr:.6f}")
print(f"Predicted CTR: {click_probability:.6f}")

print(f"\nModel confidence analysis:")
if click_probability > item_49_ctr:
    print(f"✓ Model predicts HIGHER than item 49 average (+{(click_probability-item_49_ctr)*100:.2f}%)")
else:
    print(f"✗ Model predicts LOWER than item 49 average ({(click_probability-item_49_ctr)*100:.2f}%)")

print("="*50)

Target combination to predict:
  user_feature_0: A0
  user_feature_1: A1
  user_feature_2: A2
  user_feature_3: C3
  item_id: 49

Combination exists in data: 0 observations
This is an UNSEEN combination - model will extrapolate

Encoded values:
  user_feature_0: A0 -> 0
  user_feature_1: A1 -> 0
  user_feature_2: A2 -> 0
  user_feature_3: C3 -> 2
  item_id: 49 -> 49

Prediction input shape: (1, 5)
Prediction input:
   user_feature_0  user_feature_1  user_feature_2  user_feature_3  item_id
0               0               0               0               2       49

FINAL PREDICTION RESULTS
Predicted click probability: 0.000970
Predicted class (click): 0
Confidence: 0.999030

Comparison with baselines:
Overall CTR: 0.004200
Item 49 CTR: 0.002451
Predicted CTR: 0.000970

Model confidence analysis:
✗ Model predicts LOWER than item 49 average (-0.15%)


In [99]:
# Additional Analysis: Feature Impact and Confidence Intervals
print("="*60)
print("ADDITIONAL ANALYSIS")
print("="*60)

# Check individual feature contributions by looking at similar combinations
print("\n1. PARTIAL FEATURE ANALYSIS:")
print("-" * 30)

# Look at item 49 performance with individual feature matches
item_49_data = log_df_readable[log_df_readable['item_id'] == 49]

for feature, value in target_combination.items():
    if feature != 'item_id':
        matching_data = item_49_data[item_49_data[feature] == value]
        if len(matching_data) > 0:
            ctr = matching_data['click'].mean()
            print(f"{feature}={value}: {ctr:.6f} CTR (n={len(matching_data)})")
        else:
            print(f"{feature}={value}: No data available")

print(f"\nOverall item 49 CTR: {item_49_data['click'].mean():.6f} (n={len(item_49_data)})")

# Model prediction intervals (using tree predictions)
print("\n2. PREDICTION CONFIDENCE ANALYSIS:")
print("-" * 35)

# Get predictions from individual trees for uncertainty estimation
tree_predictions = []
for estimator in rf_model.estimators_:
    pred = estimator.predict_proba(X_target)[0, 1]
    tree_predictions.append(pred)

tree_predictions = np.array(tree_predictions)
pred_mean = tree_predictions.mean()
pred_std = tree_predictions.std()
pred_ci_lower = np.percentile(tree_predictions, 5)
pred_ci_upper = np.percentile(tree_predictions, 95)

print(f"Prediction mean: {pred_mean:.6f}")
print(f"Prediction std: {pred_std:.6f}")
print(f"90% Confidence interval: [{pred_ci_lower:.6f}, {pred_ci_upper:.6f}]")
print(f"Prediction range: {pred_ci_upper - pred_ci_lower:.6f}")

# Model interpretation
print("\n3. MODEL INTERPRETATION:")
print("-" * 25)
print(f"• This combination was NOT observed in the training data")
print(f"• Model extrapolates based on similar feature patterns")
print(f"• Low prediction ({click_probability:.6f}) suggests this combination is unlikely to generate clicks")
print(f"• Prediction is below both overall CTR and item 49 CTR")
print(f"• High confidence (99.9%) in negative prediction")

# Risk assessment
risk_level = "LOW" if pred_std < 0.001 else "MEDIUM" if pred_std < 0.005 else "HIGH"
print(f"• Prediction uncertainty: {risk_level} (std: {pred_std:.6f})")

print("\n" + "="*60)

ADDITIONAL ANALYSIS

1. PARTIAL FEATURE ANALYSIS:
------------------------------
user_feature_0=A0: 0.002950 CTR (n=339)
user_feature_1=A1: 0.002890 CTR (n=346)
user_feature_2=A2: 0.015625 CTR (n=64)
user_feature_3=C3: 0.000000 CTR (n=24)

Overall item 49 CTR: 0.002451 (n=408)

2. PREDICTION CONFIDENCE ANALYSIS:
-----------------------------------
Prediction mean: 0.000970
Prediction std: 0.005122
90% Confidence interval: [0.000000, 0.000089]
Prediction range: 0.000089

3. MODEL INTERPRETATION:
-------------------------
• This combination was NOT observed in the training data
• Model extrapolates based on similar feature patterns
• Low prediction (0.000970) suggests this combination is unlikely to generate clicks
• Prediction is below both overall CTR and item 49 CTR
• High confidence (99.9%) in negative prediction
• Prediction uncertainty: HIGH (std: 0.005122)



/Users/armandoordoricadelatorre/miniconda/envs/obp_replication/lib/python3.10/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
/Users/armandoordoricadelatorre/miniconda/envs/obp_replication/lib/python3.10/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
/Users/armandoordoricadelatorre/miniconda/envs/obp_replication/lib/python3.10/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
/Users/armandoordoricadelatorre/miniconda/envs/obp_replication/lib/python3.10/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
/Users/armandoordoricadelatorre/miniconda/envs/obp_replication/lib/python3.10/site-packages/sklearn/base

In [86]:
import pandas as pd
from functools import reduce
from operator import mul

def bayes_chain_ctr(feature_tables: dict,
                    chosen_values: dict,
                    alpha_click=1.0, beta_click=1.0,
                    alpha_cond=1.0):
    """
    Estimate P(click | f0, f1, ..., fk) for a single item using a Naïve Bayes (Bayesian chain rule)
    on per-feature aggregates with Laplace smoothing.

    Parameters
    ----------
    feature_tables : dict[str, pd.DataFrame]
        For each feature name (e.g., 'user_feature_0'), a DataFrame with columns:
        ['feature_value','clicks','no_clicks','total'] covering *one item* and partitioned by that feature.
    chosen_values : dict[str, str]
        Mapping feature name -> chosen category value (e.g., {'user_feature_0':'A0', ...}).
    alpha_click, beta_click : float
        Beta prior on global CTR (for baseline odds). Defaults to Beta(1,1) Laplace.
    alpha_cond : float
        Laplace smoothing added to each category when computing P(value | click) and P(value | no click).

    Returns
    -------
    dict with:
      'p_baseline' : baseline CTR for the item (smoothed),
      'odds_baseline' : baseline odds,
      'lr_by_feature' : dict of likelihood ratios per feature,
      'odds_posterior' : combined odds,
      'p_posterior' : combined CTR estimate
    """
    # Use the first table to get global totals (all tables partition the same item)
    first = next(iter(feature_tables.values()))
    total_clicks = int(first['clicks'].sum())
    total_noclk = int(first['no_clicks'].sum())
    total_impr = total_clicks + total_noclk

    # 1) Baseline CTR with Beta(alpha_click, beta_click)
    p0 = (total_clicks + alpha_click) / (total_impr + alpha_click + beta_click)
    odds0 = p0 / (1.0 - p0)

    # 2) Per-feature likelihood ratios: P(f=value | click) / P(f=value | no-click)
    lr = {}
    for feat, df in feature_tables.items():
        # how many categories for smoothing denominator
        K = df['feature_value'].nunique()

        row = df.loc[df['feature_value'] == chosen_values[feat]]
        if row.empty:
            # unseen category for this feature: treat as 0 counts (will be handled by smoothing)
            c_clicks = 0
            c_noclk = 0
        else:
            c_clicks = int(row['clicks'].iloc[0])
            c_noclk = int(row['no_clicks'].iloc[0])

        # P(value | click)
        p_val_given_click = (c_clicks + alpha_cond) / (total_clicks + alpha_cond * K)
        # P(value | no-click)
        p_val_given_noclk = (c_noclk + alpha_cond) / (total_noclk + alpha_cond * K)

        # likelihood ratio; tiny floor to avoid division by ~0
        lr[feat] = p_val_given_click / max(p_val_given_noclk, 1e-15)

    # 3) Combine odds
    odds_post = odds0 * reduce(mul, lr.values(), 1.0)
    p_post = odds_post / (1.0 + odds_post)

    return {
        'p_baseline': p0,
        'odds_baseline': odds0,
        'lr_by_feature': lr,
        'odds_posterior': odds_post,
        'p_posterior': p_post
    }


def get_empirical_CTR_by_category(input_df, item_id, category_col):
    item_df = input_df[input_df['item_id'] == item_id]
    click_stats = item_df.groupby([category_col]).agg({
        'click': ['sum', 'count', 'mean']
    }).reset_index()

    # Flatten column names
    click_stats.columns = [category_col, 'clicks', 'total', 'click_rate']

    # Add no-clicks column
    click_stats['no_clicks'] = click_stats['total'] - click_stats['clicks']

    # Reorder columns for clarity
    click_stats = click_stats[[category_col, 'clicks', 'no_clicks', 'total', 'click_rate']]
    click_stats['item_id'] = item_id

    return click_stats

In [87]:
# Detailed click statistics by user_feature_0
N=49

user_features_list = ['user_feature_0', 'user_feature_1', 'user_feature_2', 'user_feature_3']

for feature in user_features_list:
    click_stats = get_empirical_CTR_by_category(log_df_readable, N, feature)
    print(f"\nDetailed click statistics for item_id={N} by {feature}:")
    print("=" * 60)
    print(click_stats)


Detailed click statistics for item_id=49 by user_feature_0:
  user_feature_0  clicks  no_clicks  total  click_rate  item_id
0             A0       1        338    339     0.00295       49
1             B0       0         63     63     0.00000       49
2             C0       0          6      6     0.00000       49

Detailed click statistics for item_id=49 by user_feature_1:
  user_feature_1  clicks  no_clicks  total  click_rate  item_id
0             A1       1        345    346     0.00289       49
1             B1       0         36     36     0.00000       49
2             C1       0          8      8     0.00000       49
3             D1       0         18     18     0.00000       49

Detailed click statistics for item_id=49 by user_feature_2:
  user_feature_2  clicks  no_clicks  total  click_rate  item_id
0             A2       1         63     64    0.015625       49
1             B2       0         94     94    0.000000       49
2             C2       0         92     92    0.0

In [85]:
# Build per-feature tables dynamically from the empirical data
N = 49  # item_id we're analyzing
user_features_list = ['user_feature_0', 'user_feature_1', 'user_feature_2', 'user_feature_3']

# Extract data for each feature using our function
feature_tables = {}
for feature in user_features_list:
    click_stats = get_empirical_CTR_by_category(log_df_readable, N, feature)
    
    # Convert to the format needed for bayes_chain_ctr
    feature_df = pd.DataFrame({
        'feature_value': click_stats[feature].values,
        'clicks': click_stats['clicks'].values,
        'no_clicks': click_stats['no_clicks'].values,
        'total': click_stats['total'].values,
    })
    
    feature_tables[feature] = feature_df
    
    print(f"\nFeature table for {feature}:")
    print(feature_df)

# Define the combination we want to estimate CTR for
chosen = {
    'user_feature_0': 'A0',
    'user_feature_1': 'A1', 
    'user_feature_2': 'E2',  # Changed from A2 to E2 as per your target
    'user_feature_3': 'A3',  # Changed from C3 to A3 as per your target
}

print(f"\nChosen combination: {chosen}")

# Apply Bayesian chain rule estimation
res = bayes_chain_ctr(feature_tables, chosen,
                      alpha_click=1.0, beta_click=1.0,  # Beta prior on global CTR
                      alpha_cond=1.0)                   # Laplace per-category

print("\n" + "="*60)
print("BAYESIAN CHAIN RULE RESULTS:")
print("="*60)
print(f"Baseline CTR (smoothed): {res['p_baseline']:.6f}")
print(f"Likelihood ratios by feature:")
for feat, lr in res['lr_by_feature'].items():
    print(f"  {feat}: {lr:.6f}")
print(f"Posterior CTR estimate: {res['p_posterior']:.6f}")
print("="*60)


Feature table for user_feature_0:
  feature_value  clicks  no_clicks  total
0            A0       1        338    339
1            B0       0         63     63
2            C0       0          6      6

Feature table for user_feature_1:
  feature_value  clicks  no_clicks  total
0            A1       1        345    346
1            B1       0         36     36
2            C1       0          8      8
3            D1       0         18     18

Feature table for user_feature_2:
  feature_value  clicks  no_clicks  total
0            A2       1         63     64
1            B2       0         94     94
2            C2       0         92     92
3            D2       0         60     60
4            E2       0         69     69
5            F2       0         21     21
6            G2       0          8      8

Feature table for user_feature_3:
  feature_value  clicks  no_clicks  total
0            A3       1        154    155
1            B3       0        146    146
2            C3     

In [ ]:
# Define the target: item 49 with user features (A0, A1, E2, A3)
target_item_49 = 49
target_features_49 = ['A0', 'A1', 'E2', 'A3']

print(f"Target: item_id={target_item_49}, user_features={target_features_49}")

# Check if this combination exists in the dataset
exists_49 = log_df_readable[
    (log_df_readable['item_id'] == target_item_49) &
    (log_df_readable['user_feature_0'] == target_features_49[0]) &
    (log_df_readable['user_feature_1'] == target_features_49[1]) &
    (log_df_readable['user_feature_2'] == target_features_49[2]) &
    (log_df_readable['user_feature_3'] == target_features_49[3])
]

print(f"\nObservations with this exact combination: {len(exists_49)}")
if len(exists_49) > 0:
    print(f"Observed CTR: {exists_49['click'].mean():.4f}")
else:
    print("This is an UNOBSERVED combination - we need to estimate CTR")

In [ ]:
### Method 1: Marginal Averaging for Item 49

# Filter to only item 49
item_49_data = log_df_readable[log_df_readable['item_id'] == target_item_49].copy()

print(f"Total observations for item {target_item_49}: {len(item_49_data)}")
print(f"Overall CTR for item {target_item_49}: {item_49_data['click'].mean():.4f}")

# Calculate CTR for different levels of feature matching
marginal_estimates_49 = {}

# Overall item CTR (no feature matching)
marginal_estimates_49['overall_item'] = item_49_data['click'].mean()

# Match each individual feature
for i, feat_val in enumerate(target_features_49):
    feat_name = f'user_feature_{i}'
    matching_data = item_49_data[item_49_data[feat_name] == feat_val]
    if len(matching_data) > 0:
        marginal_estimates_49[f'{feat_name}={feat_val}'] = matching_data['click'].mean()
        print(f"CTR when {feat_name}={feat_val}: {marginal_estimates_49[f'{feat_name}={feat_val}']:.4f} (n={len(matching_data)})")
    else:
        marginal_estimates_49[f'{feat_name}={feat_val}'] = np.nan
        print(f"CTR when {feat_name}={feat_val}: NO DATA")

# Match pairs of features
feature_pairs_49 = [
    (0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)
]

for i, j in feature_pairs_49:
    feat_i, feat_j = f'user_feature_{i}', f'user_feature_{j}'
    val_i, val_j = target_features_49[i], target_features_49[j]
    matching_data = item_49_data[
        (item_49_data[feat_i] == val_i) & 
        (item_49_data[feat_j] == val_j)
    ]
    if len(matching_data) > 0:
        key = f'{feat_i}={val_i}, {feat_j}={val_j}'
        marginal_estimates_49[key] = matching_data['click'].mean()
        print(f"CTR when {key}: {marginal_estimates_49[key]:.4f} (n={len(matching_data)})")

# Match triples
feature_triples_49 = [
    (0, 1, 2), (0, 1, 3), (0, 2, 3), (1, 2, 3)
]

for i, j, k in feature_triples_49:
    feat_i, feat_j, feat_k = f'user_feature_{i}', f'user_feature_{j}', f'user_feature_{k}'
    val_i, val_j, val_k = target_features_49[i], target_features_49[j], target_features_49[k]
    matching_data = item_49_data[
        (item_49_data[feat_i] == val_i) & 
        (item_49_data[feat_j] == val_j) & 
        (item_49_data[feat_k] == val_k)
    ]
    if len(matching_data) > 0:
        key = f'{feat_i}={val_i}, {feat_j}={val_j}, {feat_k}={val_k}'
        marginal_estimates_49[key] = matching_data['click'].mean()
        print(f"CTR when {key}: {marginal_estimates_49[key]:.4f} (n={len(matching_data)})")

# Summary
print("\n" + "="*60)
print("MARGINAL AVERAGING ESTIMATES (Item 49):")
print("="*60)
for key, val in marginal_estimates_49.items():
    if not np.isnan(val):
        print(f"{key:50s}: {val:.4f}")

In [ ]:
### Method 2: Random Forest Classifier for Item 49

# Prepare features for all data
feature_cols = ['user_feature_0', 'user_feature_1', 'user_feature_2', 'user_feature_3', 'item_id']

# Encode categorical features
from sklearn.preprocessing import LabelEncoder

le_dict_49 = {}
X_encoded = log_df_readable[feature_cols].copy()

for col in feature_cols:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(log_df_readable[col])
    le_dict_49[col] = le

y = log_df_readable['click']

# Train Random Forest
from sklearn.ensemble import RandomForestClassifier

rf_model_49 = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf_model_49.fit(X_encoded, y)

# Create target observation for prediction
target_obs_49 = pd.DataFrame({
    'user_feature_0': [target_features_49[0]],
    'user_feature_1': [target_features_49[1]],
    'user_feature_2': [target_features_49[2]],
    'user_feature_3': [target_features_49[3]],
    'item_id': [target_item_49]
})

# Encode target observation
target_encoded_49 = target_obs_49.copy()
for col in feature_cols:
    target_encoded_49[col] = le_dict_49[col].transform(target_obs_49[col])

# Predict probability of click
rf_pred_proba_49 = rf_model_49.predict_proba(target_encoded_49)[0, 1]

print(f"Random Forest predicted CTR for item {target_item_49} with features {target_features_49}:")
print(f"  CTR = {rf_pred_proba_49:.4f}")

# Show feature importances
print("\nFeature Importances:")
for col, importance in zip(feature_cols, rf_model_49.feature_importances_):
    print(f"  {col:20s}: {importance:.4f}")

In [ ]:
### Method 3: Direct Method (DM) for Item 49

# The DM estimator uses a reward model to predict CTR
# We'll use the same Random Forest model as the reward model for DM

dm_estimate_49 = rf_pred_proba_49  # Same as RF prediction

print(f"Direct Method (DM) estimated CTR for item {target_item_49}:")
print(f"  CTR = {dm_estimate_49:.4f}")
print("\nNote: DM uses the reward model (Random Forest) to predict expected reward")

In [ ]:
### Method 4: Doubly Robust (DR) for Item 49

# For an UNOBSERVED combination, DR reduces to DM because there's no
# observational data to provide the IPS correction term

# DR formula: E[R] = E[DM(x,a)] + E[IPS * (R - DM(x,a))]
# For unobserved: second term = 0 (no matching observations)

dr_estimate_49 = dm_estimate_49  # Reduces to DM for unobserved

print(f"Doubly Robust (DR) estimated CTR for item {target_item_49}:")
print(f"  CTR = {dr_estimate_49:.4f}")
print("\nNote: For unobserved combinations, DR reduces to DM")
print("(no IPS correction possible without observed data)")

In [ ]:
### Summary: All Methods for Item 49

# Compile all estimates
estimates_49 = {
    'Overall Item CTR': marginal_estimates_49['overall_item'],
    'Random Forest': rf_pred_proba_49,
    'Direct Method (DM)': dm_estimate_49,
    'Doubly Robust (DR)': dr_estimate_49
}

# Add marginal averages (excluding overall)
for key, val in marginal_estimates_49.items():
    if key != 'overall_item' and not np.isnan(val):
        estimates_49[f'Marginal: {key}'] = val

print("="*80)
print(f"CTR ESTIMATES FOR ITEM {target_item_49} WITH FEATURES {target_features_49}")
print("="*80)

for method, estimate in estimates_49.items():
    print(f"{method:50s}: {estimate:.4f}")

print("="*80)

In [ ]:
### Visualization: Comparison of Estimation Methods (Item 49)

# Prepare data for visualization
main_methods_49 = {
    'Overall\nItem CTR': marginal_estimates_49['overall_item'],
    'Random\nForest': rf_pred_proba_49,
    'Direct\nMethod': dm_estimate_49,
    'Doubly\nRobust': dr_estimate_49
}

# Create comparison plot
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Main methods
ax1 = axes[0]
methods = list(main_methods_49.keys())
values = list(main_methods_49.values())
colors = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12']

bars = ax1.bar(methods, values, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
ax1.set_ylabel('Estimated CTR', fontsize=12, fontweight='bold')
ax1.set_title(f'CTR Estimates for Item {target_item_49}\nUser Features: {target_features_49}', 
              fontsize=14, fontweight='bold')
ax1.set_ylim(0, max(values) * 1.2)
ax1.grid(axis='y', alpha=0.3, linestyle='--')

# Add value labels on bars
for bar, val in zip(bars, values):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{val:.4f}',
             ha='center', va='bottom', fontsize=11, fontweight='bold')

# Plot 2: All marginal averages
ax2 = axes[1]
marginal_keys = [k for k in marginal_estimates_49.keys() if not np.isnan(marginal_estimates_49[k])]
marginal_vals = [marginal_estimates_49[k] for k in marginal_keys]

# Shorten labels for readability
short_labels = []
for key in marginal_keys:
    if key == 'overall_item':
        short_labels.append('Overall')
    else:
        # Extract feature matches
        short_labels.append(key.replace('user_feature_', 'F').replace(', user_feature_', ', F'))

bars2 = ax2.barh(short_labels, marginal_vals, color='#9b59b6', alpha=0.7, edgecolor='black', linewidth=1.5)
ax2.set_xlabel('Estimated CTR', fontsize=12, fontweight='bold')
ax2.set_title('Marginal Averaging Estimates\n(Different Feature Match Levels)', 
              fontsize=14, fontweight='bold')
ax2.set_xlim(0, max(marginal_vals) * 1.15)
ax2.grid(axis='x', alpha=0.3, linestyle='--')
ax2.invert_yaxis()  # Highest match level at top

# Add value labels
for bar, val in zip(bars2, marginal_vals):
    width = bar.get_width()
    ax2.text(width, bar.get_y() + bar.get_height()/2.,
             f' {val:.4f}',
             ha='left', va='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\n✓ Visualization complete for item {target_item_49}")

In [ ]:
# Example: Show a specific missing combination
if len(missing_combos) > 0:
    example_missing = missing_combos.iloc[0]
    print("Example of a missing combination:")
    print(example_missing.to_dict())
    
    # Verify it's not in the data
    query = ' & '.join([f"(log_df_readable['{col}'] == '{val}')" 
                        for col, val in example_missing.to_dict().items()])
    query += " & (log_df_readable['item_id'] == 0)"
    
    result = eval(f"log_df_readable[{query}]")
    print(f"\nRows matching this combination with item_id=0: {len(result)}")
else:
    print("All possible combinations are present in the data!")

### Estimating CTR for Unseen User-Item Combinations

**Problem:** We want to estimate the click-through probability for item_id=0 with user features (A0, A1, A2, D3), which has never been observed in the logged data.

**Approaches:**
1. **Marginal Averaging**: Average CTR from similar partial matches
2. **Feature-based Model**: Train a supervised model (e.g., Random Forest, Logistic Regression)
3. **Matrix Factorization/Embeddings**: Learn latent representations
4. **Nearest Neighbor**: Find similar observed combinations

In [ ]:
# Define the unseen combination we want to estimate CTR for
target_combo = {
    'user_feature_0': 'A0',
    'user_feature_1': 'A1', 
    'user_feature_2': 'A2',
    'user_feature_3': 'D3'
}
target_item = 0

print(f"Target: Estimate CTR for item_id={target_item} with features:")
for k, v in target_combo.items():
    print(f"  {k}: {v}")

# Verify this combination doesn't exist for item_id=0
query_result = log_df_readable[
    (log_df_readable['item_id'] == target_item) &
    (log_df_readable['user_feature_0'] == target_combo['user_feature_0']) &
    (log_df_readable['user_feature_1'] == target_combo['user_feature_1']) &
    (log_df_readable['user_feature_2'] == target_combo['user_feature_2']) &
    (log_df_readable['user_feature_3'] == target_combo['user_feature_3'])
]

print(f"\nObservations matching this exact combination: {len(query_result)}")
if len(query_result) == 0:
    print("✓ Confirmed: This is an unseen combination")

#### Approach 1: Marginal Averaging from Partial Matches

Find similar combinations by matching subsets of features and average their CTRs.

In [ ]:
# Approach 1: Marginal Averaging
# Find partial matches and compute weighted average

item_0_data = log_df_readable[log_df_readable['item_id'] == target_item].copy()

estimates = {}

# 1. Overall average for item_id=0 (baseline)
estimates['overall_item_avg'] = item_0_data['click'].mean()

# 2. Average for each individual feature match
for feat, val in target_combo.items():
    mask = item_0_data[feat] == val
    if mask.sum() > 0:
        estimates[f'match_{feat}'] = item_0_data[mask]['click'].mean()
        estimates[f'n_{feat}'] = mask.sum()

# 3. Average for combinations of 2 features
estimates['match_f0_f1'] = item_0_data[
    (item_0_data['user_feature_0'] == target_combo['user_feature_0']) &
    (item_0_data['user_feature_1'] == target_combo['user_feature_1'])
]['click'].mean() if len(item_0_data[
    (item_0_data['user_feature_0'] == target_combo['user_feature_0']) &
    (item_0_data['user_feature_1'] == target_combo['user_feature_1'])
]) > 0 else np.nan

# 4. Average for combinations of 3 features  
estimates['match_f0_f1_f2'] = item_0_data[
    (item_0_data['user_feature_0'] == target_combo['user_feature_0']) &
    (item_0_data['user_feature_1'] == target_combo['user_feature_1']) &
    (item_0_data['user_feature_2'] == target_combo['user_feature_2'])
]['click'].mean() if len(item_0_data[
    (item_0_data['user_feature_0'] == target_combo['user_feature_0']) &
    (item_0_data['user_feature_1'] == target_combo['user_feature_1']) &
    (item_0_data['user_feature_2'] == target_combo['user_feature_2'])
]) > 0 else np.nan

print("Marginal Averaging Estimates:")
print("="*50)
for key, val in estimates.items():
    if not key.startswith('n_'):
        print(f"{key:30s}: {val:.4f}" if not np.isnan(val) else f"{key:30s}: N/A")

# Final estimate: Use the most specific available estimate
marginal_estimate = estimates.get('match_f0_f1_f2')
if np.isnan(marginal_estimate) or marginal_estimate is None:
    marginal_estimate = estimates.get('match_f0_f1')
if np.isnan(marginal_estimate) or marginal_estimate is None:
    marginal_estimate = np.nanmean([estimates.get(f'match_{k}', np.nan) for k in target_combo.keys()])
if np.isnan(marginal_estimate):
    marginal_estimate = estimates['overall_item_avg']

print(f"\n{'Final Marginal Estimate':30s}: {marginal_estimate:.4f}")

#### Approach 2: Supervised Learning Model (Random Forest)

Train a model to predict CTR based on user features and item_id, then use it to predict for the unseen combination.

In [ ]:
# Approach 2: Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# Prepare data for modeling
model_data = log_df_readable.copy()

# Encode categorical features
le_dict = {}
for col in user_feature_cols:
    le = LabelEncoder()
    model_data[f'{col}_encoded'] = le.fit_transform(model_data[col].astype(str))
    le_dict[col] = le

# Features and target
feature_cols = [f'{col}_encoded' for col in user_feature_cols] + ['item_id', 'position']
X = model_data[feature_cols]
y = model_data['click']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest
rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Evaluate on test set
train_acc = rf_model.score(X_train, y_train)
test_acc = rf_model.score(X_test, y_test)
print(f"Random Forest Performance:")
print(f"  Train Accuracy: {train_acc:.4f}")
print(f"  Test Accuracy: {test_acc:.4f}")

# Predict for the unseen combination
target_features_encoded = {}
for col in user_feature_cols:
    try:
        target_features_encoded[f'{col}_encoded'] = le_dict[col].transform([target_combo[col]])[0]
    except:
        # If the value wasn't seen during training, use the most common value
        target_features_encoded[f'{col}_encoded'] = 0
        print(f"Warning: {target_combo[col]} not seen in training for {col}")

# Create prediction input for all positions
predictions_by_position = {}
for pos in [1, 2, 3]:
    X_pred = pd.DataFrame([{
        'user_feature_0_encoded': target_features_encoded['user_feature_0_encoded'],
        'user_feature_1_encoded': target_features_encoded['user_feature_1_encoded'],
        'user_feature_2_encoded': target_features_encoded['user_feature_2_encoded'],
        'user_feature_3_encoded': target_features_encoded['user_feature_3_encoded'],
        'item_id': target_item,
        'position': pos
    }])
    
    # Predict probability of click
    pred_proba = rf_model.predict_proba(X_pred)[0, 1]  # Probability of class 1 (click)
    predictions_by_position[f'position_{pos}'] = pred_proba

print(f"\nRandom Forest CTR Estimates for item_id={target_item}:")
print(f"  Target combination: {target_combo}")
for pos, prob in predictions_by_position.items():
    print(f"  {pos}: {prob:.4f}")

# Average across positions (if position-agnostic estimate needed)
rf_estimate = np.mean(list(predictions_by_position.values()))
print(f"\n{'Final RF Estimate (avg)':30s}: {rf_estimate:.4f}")

In [ ]:
# Summary: Compare all estimates
print("="*60)
print("SUMMARY: CTR Estimates for Unseen Combination")
print("="*60)
print(f"Item ID: {target_item}")
print(f"User Features: {target_combo}")
print("\nEstimates:")
print(f"  1. Marginal Averaging: {marginal_estimate:.4f}")
print(f"  2. Random Forest:      {rf_estimate:.4f}")
print(f"\n  Baseline (overall item avg): {estimates['overall_item_avg']:.4f}")
print("="*60)

# Create a comparison DataFrame
comparison_df = pd.DataFrame({
    'Method': ['Marginal Averaging', 'Random Forest', 'Baseline (Item Avg)'],
    'Estimated_CTR': [marginal_estimate, rf_estimate, estimates['overall_item_avg']]
})

# Visualize
fig = px.bar(comparison_df, 
             x='Method', 
             y='Estimated_CTR',
             title=f'CTR Estimates for Unseen Combination (item_id={target_item})',
             labels={'Estimated_CTR': 'Estimated CTR'},
             text='Estimated_CTR',
             height=400)

fig.update_traces(texttemplate='%{text:.4f}', textposition='outside')
fig.show()

#### Approach 3: Offline Bandit Evaluation Methods

Use proper OPE (Offline Policy Evaluation) estimators that leverage propensity scores:
- **IPS (Inverse Propensity Scoring)**: Reweights logged data by inverse propensity
- **DR (Doubly Robust)**: Combines IPS with a reward model for lower variance
- **DM (Direct Method)**: Uses a learned reward model only

These methods are designed specifically for counterfactual estimation in contextual bandits.

In [ ]:
# Step 1: Build a regression model to estimate E[r|x,a] (expected reward given context and action)
# This will be used for Direct Method (DM) and Doubly Robust (DR) estimators

from sklearn.ensemble import RandomForestRegressor

# Prepare data - use the same encoding as before
reward_model_data = log_df_readable.copy()

# Use the already encoded features from before
for col in user_feature_cols:
    reward_model_data[f'{col}_encoded'] = le_dict[col].transform(reward_model_data[col].astype(str))

# Features: user features + item_id + position
reward_feature_cols = [f'{col}_encoded' for col in user_feature_cols] + ['item_id', 'position']
X_reward = reward_model_data[reward_feature_cols]
y_reward = reward_model_data['click']

# Train reward model (regression for expected reward)
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_reward, y_reward, test_size=0.2, random_state=42)

reward_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
reward_model.fit(X_train_r, y_train_r)

train_r2 = reward_model.score(X_train_r, y_train_r)
test_r2 = reward_model.score(X_test_r, y_test_r)

print(f"Reward Model Performance (R²):")
print(f"  Train: {train_r2:.4f}")
print(f"  Test:  {test_r2:.4f}")

# Get estimated rewards for all data (q-hat)
reward_model_data['estimated_reward'] = reward_model.predict(X_reward)

print(f"\nEstimated reward distribution:")
print(reward_model_data['estimated_reward'].describe())

In [ ]:
# Direct Method (DM): Simply use the reward model prediction
# E[r|x,a] where x = target_combo, a = target_item

dm_estimates = {}
for pos in [1, 2, 3]:
    X_target = pd.DataFrame([{
        'user_feature_0_encoded': target_features_encoded['user_feature_0_encoded'],
        'user_feature_1_encoded': target_features_encoded['user_feature_1_encoded'],
        'user_feature_2_encoded': target_features_encoded['user_feature_2_encoded'],
        'user_feature_3_encoded': target_features_encoded['user_feature_3_encoded'],
        'item_id': target_item,
        'position': pos
    }])
    
    dm_estimates[f'position_{pos}'] = reward_model.predict(X_target)[0]

print(f"Direct Method (DM) Estimates:")
print(f"  Target: item_id={target_item}, features={target_combo}")
for pos, est in dm_estimates.items():
    print(f"  {pos}: {est:.4f}")

dm_estimate = np.mean(list(dm_estimates.values()))
print(f"\n{'Final DM Estimate (avg)':30s}: {dm_estimate:.4f}")

In [ ]:
# Doubly Robust (DR) Estimator
# For unseen combinations, we can't use IPS directly (no logged data with this exact combo)
# But we can understand how DR would work if we had similar data

# DR formula: E[V(π)] = E[q(x,a) + (r - q(x,a)) * π(a|x) / π_b(a|x)]
# where:
#   - q(x,a) is the estimated reward model (already have this)
#   - r is the actual reward (observed)
#   - π(a|x) is the evaluation policy (what we'd like to evaluate)
#   - π_b(a|x) is the behavior policy (propensity scores we have)

# For our unseen combination, we only have q(x,a) since there's no logged data
# So DR reduces to DM for completely unseen combinations

print("Doubly Robust (DR) Estimator:")
print("="*60)
print(f"For UNSEEN combinations (like our target):")
print(f"  - No logged data exists → IPS term = 0")
print(f"  - DR reduces to Direct Method (DM)")
print(f"  - Estimate = q(x,a) = {dm_estimate:.4f}")
print()
print(f"For SEEN combinations:")
print(f"  - DR = q(x,a) + (r - q(x,a)) * π(a|x) / π_b(a|x)")
print(f"  - Combines model prediction with importance-weighted residuals")
print(f"  - Lower variance than pure IPS, less biased than pure DM")
print("="*60)

# Example: Show DR for a SEEN combination for comparison
seen_example = item_0_data.iloc[0]  # First row with item_id=0

# Get the reward model prediction for this seen example
seen_features = {
    'user_feature_0_encoded': le_dict['user_feature_0'].transform([seen_example['user_feature_0']])[0],
    'user_feature_1_encoded': le_dict['user_feature_1'].transform([seen_example['user_feature_1']])[0],
    'user_feature_2_encoded': le_dict['user_feature_2'].transform([seen_example['user_feature_2']])[0],
    'user_feature_3_encoded': le_dict['user_feature_3'].transform([seen_example['user_feature_3']])[0],
    'item_id': int(seen_example['item_id']),
    'position': int(seen_example['position'])
}

X_seen = pd.DataFrame([seen_features])
q_hat = reward_model.predict(X_seen)[0]
actual_reward = seen_example['click']
propensity = seen_example['propensity_score']

# Assuming evaluation policy would always select this item (π(a|x) = 1)
eval_policy_prob = 1.0

# DR estimate for this one example
dr_contribution = q_hat + (actual_reward - q_hat) * (eval_policy_prob / propensity)

print(f"\nExample DR calculation for a SEEN combination:")
print(f"  Features: {seen_example[user_feature_cols].to_dict()}")
print(f"  q̂(x,a):         {q_hat:.4f}")
print(f"  actual reward:  {actual_reward:.4f}")
print(f"  propensity:     {propensity:.4f}")
print(f"  DR estimate:    {dr_contribution:.4f}")

In [ ]:
# Using OBP's OffPolicyEvaluation framework
# This demonstrates how to evaluate a NEW policy that prefers our target combination

from obp.ope import (
    OffPolicyEvaluation,
    InverseProbabilityWeighting as IPW,
    DirectMethod as DM_OBP,
    DoublyRobust as DR_OBP,
    RegressionModel
)

print("Using OBP for Policy Evaluation:")
print("="*60)

# Create action distributions for evaluation policy
# We'll create a policy that has higher probability for item_id=0 with our target features

# For OBP, we need action_dist in format: (n_rounds, n_actions, len_list)
# But for this demo, let's show the conceptual approach

print("\nKey Insights for Unseen Combinations:")
print("-" * 60)
print("1. IPS (Inverse Propensity Scoring):")
print("   - Requires: logged data with π_b(a|x) > 0")
print("   - Cannot estimate for unseen combinations directly")
print("   - Unbiased but high variance")
print()
print("2. Direct Method (DM):")
print("   - Uses: q̂(x,a) from regression model")
print("   - CAN estimate for unseen combinations ✓")
print("   - Biased if model is wrong, but lower variance")
print(f"   - Our estimate: {dm_estimate:.4f}")
print()
print("3. Doubly Robust (DR):")
print("   - Combines: q̂(x,a) + IPS correction")
print("   - For unseen: reduces to DM (no IPS correction possible)")
print(f"   - Our estimate: {dm_estimate:.4f}")
print()
print("4. Propensity Scores:")
print(f"   - Behavior policy propensity for item {target_item}:")
print(f"     Average: {log_df_readable[log_df_readable['item_id']==target_item]['propensity_score'].mean():.4f}")
print(f"     But for our EXACT combo: 0 (never shown)")
print("="*60)

In [ ]:
# Final Comprehensive Comparison
print("\n" + "="*70)
print("COMPREHENSIVE COMPARISON: All Methods")
print("="*70)
print(f"Target: item_id={target_item}")
print(f"Features: {target_combo}")
print(f"Status: UNSEEN combination (0 logged observations)")
print()
print(f"{'Method':<30} {'Estimate':<12} {'Notes'}")
print("-"*70)
print(f"{'1. Marginal Averaging':<30} {marginal_estimate:>10.4f}   Heuristic")
print(f"{'2. Random Forest (Direct)':<30} {rf_estimate:>10.4f}   Supervised ML")
print(f"{'3. DM (Reward Model)':<30} {dm_estimate:>10.4f}   Model-based")
print(f"{'4. DR (reduces to DM)':<30} {dm_estimate:>10.4f}   No IPS term")
print(f"{'5. Baseline (Item Avg)':<30} {estimates['overall_item_avg']:>10.4f}   Simple avg")
print("="*70)
print()
print("Recommendations:")
print("• Use DM/DR with a good reward model for unseen combinations")
print("• Marginal averaging is interpretable but may miss interactions")  
print("• For SEEN combinations, DR > IPS > DM (typically)")
print("• For UNSEEN combinations, DM is the only propensity-based option")
print("="*70)

# Updated visualization
comparison_df_full = pd.DataFrame({
    'Method': [
        'Marginal\nAveraging', 
        'Random Forest\n(Classification)',
        'Direct Method\n(Regression)',
        'Doubly Robust\n(DR→DM)',
        'Baseline\n(Item Avg)'
    ],
    'Estimated_CTR': [
        marginal_estimate, 
        rf_estimate, 
        dm_estimate,
        dm_estimate,  # Same as DM for unseen
        estimates['overall_item_avg']
    ],
    'Type': [
        'Heuristic',
        'Supervised ML',
        'OPE (Model)',
        'OPE (Hybrid)',
        'Baseline'
    ]
})

fig = px.bar(comparison_df_full, 
             x='Method', 
             y='Estimated_CTR',
             color='Type',
             title=f'CTR Estimates for Unseen Combination<br>item_id={target_item}, features={target_combo}',
             labels={'Estimated_CTR': 'Estimated CTR'},
             text='Estimated_CTR',
             height=500)

fig.update_traces(texttemplate='%{text:.4f}', textposition='outside')
fig.update_layout(showlegend=True, xaxis_tickangle=-15)
fig.show()

In [ ]:
log_df_readable[log_df_readable['item_id'] == 0].groupby('propensity_score').size().reset_index(name='count').sort_values('propensity_score').sort_values(by='count', ascending=False)

### Manual Propensity Score Computation

This function computes manual propensity scores based on the empirical frequency of (item, categorical_feature) pairs in the logged data.

**Use cases:**
- Compute propensity scores for any combination of item_id and categorical features (user_feature_0, position, etc.)
- The propensity represents P(item, feature) based on observed counts
- Can be used to understand the logging policy's behavior

In [ ]:
sns.distplot(log_df_readable[log_df_readable['item_id'] == 0]['propensity_score'], bins=30, kde=False)

In [ ]:
log_df_readable.head()

In [ ]:
log_df.groupby("item_id")["propensity_score"].agg(["mean", "std"]).head(10)